# **Resumo de Podcast para Ouvintes Ocupados**

Bem-vindo ao projeto que faz parte da masterclass - **Do Conceito à Realidade: Crie seus próprios produtos de IA com ChatGPT** oferecida pelo Dr. Jorge Valverde-Rebaza (detalhes [AQUI](https://visibilia.net.br/masterclass-ia-generativa-crie-produtos-genai-chatgpt/)). Neste projeto, você criará um aplicativo LLM que resume um episódio de podcast, identifica convidados do podcast, identifica os principais destaques analisando o conteúdo do mesmo, e muito mais!

## **Problema**

Sou um grande fã de podcasts e adoro podcasts por serem uma forma de obter *insights* profundos sobre diferentes tópicos, incluindo negócios, tecnologia, e inclusive saúde. Os podcasts ajudam a aprender com a experiência vivida de pessoas em todo o mundo. Mas o tempo é limitado e acabo ouvindo apenas alguns deles! Estou inscrito em vários podcasts interessantes que lançam 1-2 episódios por semana e o problema é que é difícil identificar um episódio que me agrade. Embora muitos episódios forneçam notas de apresentação, links adicionais e carimbos de data/hora, tais informações terminam sendo pouco úteis para realmente entender os aspectos únicos do podcast e determinar se vou querer ouvi-los! Como posso tornar esse processo de descoberta mais interessante e eficiente?

## **Solução**

Pode-se gerar um boletim informativo personalizado toda semana que resuma cada episódio de podcast lançado naquela semana. Incluiria informações sobre o convidado no episódio, os principais tópicos discutidos, bem como alguns destaques. Para fazer isso possível, deve-se coletar uma lista de feeds RSS do usuário e, periodicamente, processar os episódios mais recentes para gerar o boletim informativo. Isso funciona como um resumo da semana, fornecendo ao usuário o nível certo de detalhes que lhe permitira decidir quais episódios são atraentes e devem ser ouvidos.

## **Abordagem**

Dividiremos a abordagem para construir este produto em três partes -

- **Parte 1:** usar um modelo de Speech to Text da OpenAI chamado Whisper para transcrever o podcast.

- **Parte 2:** usar um LLM da OpenAI para construir a funcionalidade de extração de informações para obter insights do podcast.

- **Parte 3:** usar o chatGPT da OpenAI como seu assistente de codificação para criar e implantar um front-end que permita aos usuários experimentar a funcionalidade de ponta a ponta

Por meio deste projeto, nosso objetivo é entender como podemos abordar a construção e a implantação de aplicativos LLM que agregam valor aos usuários. Também forneceremos ideias para extensões interessantes e funcionalidades adicionais em várias partes do projeto.

In [1]:
#installing libraries
!pip uninstall typing-extensions -y
!pip uninstall gradio -y
!pip install typing_extensions==4.7.1
!pip install gradio
!pip install feedparser
!pip install transformers accelerate
!pip install openai
!pip install tiktoken
!pip install wikipedia

Found existing installation: typing_extensions 4.12.2
Uninstalling typing_extensions-4.12.2:
  Successfully uninstalled typing_extensions-4.12.2
Found existing installation: gradio 4.36.1
Uninstalling gradio-4.36.1:
  Successfully uninstalled gradio-4.36.1
  Using cached typing_extensions-4.7.1-py3-none-any.whl (33 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.111.0 requires typing-extensions>=4.8.0, but you have typing-extensions 4.7.1 which is incompatible.
torch 2.3.0+cu121 requires typing-extensions>=4.8.0, but you have typing-extensions 4.7.1 which is incompatible.
  Using cached gradio-4.36.1-py3-none-any.whl (12.3 MB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully

In [2]:
#calling packages

import feedparser
import tiktoken
import json
import os
import wikipedia
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from openai import OpenAI
from google.colab import userdata
from huggingface_hub import notebook_login
import gradio as gr

# **I. Podcast Transcription**



## **1.1 Collecting Audio Data File**

O primeiro passo é obter os dados dos podcast do nosso interesse e coletar tais dados. Uma maneira eficaz para fazermos isso é:


1.   Acessar o *feed RSS* do podcast do nosso interesse
2.   Identificar o episódio de interesse
3.   Recuperar o arquivo de áudio (normalmente no formato MP3).

Um dos sites mais comuns para encontrar informações sobre podcasts é o [ListenNotes](https://www.listennotes.com/). Você pode pesquisar qualquer podcast que goste e clicar no ícone RSS para recuperar o feed RSS. Por exemplo, The Economist publica um podcast diário de resumo de notícias chamado [The Intelligence](https://www.listennotes.com/podcasts/the-intelligence-from-the-economist-the-stZlS3fr8KO/) que é bem conciso e informativo. Vamos usar o feed RSS deste podcast.

<br>



---




**Considerações importantes**

Devido a que trabalharemos com recursos computacionais limitados, devemos ter em conta que:

*   Trabalharemos com podcasts que tenham duração máxima de 25 a 30 minutos

Para trabalhar com podcasts de tamanho maior, é possível que além de precisar do uso de GPUs, haja também um consumo maior na API de OpenAI bem como a necessidade do uso de estratégias para o particionamento do tamanho do token de entraa/saída em múltiplas chamadas à API de OpenAI.
<br>


---



Começamos por acessar o podcast por completo

In [3]:
podcast_url = "https://access.acast.com/rss/d556eb54-6160-4c85-95f4-47d9f5216c49"
podcast_feed = feedparser.parse(podcast_url)
print ("The number of podcast entries is ", len(podcast_feed.entries))

The number of podcast entries is  1412


In [4]:
podcast_title = podcast_feed['feed']['title']
print("The podcast tiles is ", podcast_title)

The podcast tiles is  The Intelligence from The Economist


In [5]:
episode_title = podcast_feed.entries[0]['title']
print("The title of podcast episode is ", episode_title)


The title of podcast episode is  Empire of the sun: a solar power revolution


In [6]:
episode_image = podcast_feed['feed']['image'].href
print("The url of episode image is: ", episode_image)

The url of episode image is:  https://assets.pippa.io/shows/62e286a934d4d93d6587424a/1708532528162-b9341a1e28f72ef144e53cf135fce49e.jpeg


Vamos obter a URL do episódio da nossa preferência. No nosso caso, ficaremos com a entrada mais recente do feed.

Logo, faremos o download do arquivo MP3 correspondente e o salvaremos no próprio Google Colab com o nome `my_favourite_podcast.mp3`

In [7]:
for item in podcast_feed.entries[0].links:
  if (item['type'] == 'audio/mpeg'):
    my_episode = item.href
!wget -O 'my_favourite_podcast.mp3' {my_episode}

--2024-06-20 18:17:49--  https://sphinx.acast.com/p/acast/s/theintelligencepodcast/e/6674023ffcc9a9001331f5ee/media.mp3?tk=eyJ0ayI6ImRlZmF1bHQiLCJhZHMiOnRydWUsInNwb25zIjp0cnVlLCJzdGF0dXMiOiJwdWJsaWMifQ==
Resolving sphinx.acast.com (sphinx.acast.com)... 13.226.210.97, 13.226.210.118, 13.226.210.122, ...
Connecting to sphinx.acast.com (sphinx.acast.com)|13.226.210.97|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://stitcher2.acast.com/livestitches/4b568a517f3221d7302f0fb9b178f578.mp3?aid=6674023ffcc9a9001331f5ee&chid=d556eb54-6160-4c85-95f4-47d9f5216c49&ci=spxpu10QI2KVj_s8OD3JH_hVYr_FDSsS7oVPkWxv72Lbf3DoPI6ElQ%3D%3D&pf=rss&sv=sphinx%401.208.0&uid=96e73331586fd8e61ce2ace636960b2a&Expires=1718919169864&Key-Pair-Id=K38CTQXUSD0VVB&Signature=LizO2G589VLcGNyrtuXtq6mJoLYTySrDlw-~xlddIblwgeeSUEzIO6MXV-G7VzSM7x9WCpZG7usooAOaAGEttnN1G6QK5t9g1jK7ZGjhaDDGpmnTMRmN5yrKrINrv~MCv87lh6RM3WIz2SS-HH9jtk5pZVgGmUX73lr7CrirfG3Z1CPJ71Dye8qt-Ohtl76~YR9TwRPiTM7sK9k7IoRrgsnnq

Para confirmar se o episódio do podcast foi baixado com sucesso, você pode usar o painel de navegação esquerdo no Google Colab para navegar até a seção de pastas e verificar.

## **1.2 Transcribe the Audio File**

Usaremos [Distill Whisper](https://huggingface.co/distil-whisper/distil-medium.en) como nosso modelo de fala para texto. O modelo original foi disponibilizado pela OpenAI e esta versão foi destilada (compactada) usando ajustes finos adicionais para que funcione mais rápido. Há uma ligeira queda na precisão, mas para nosso caso de uso isso não terá um grande impacto.

Para usar o HuggingFace, é necessário primeiro criar uma [conta](https://huggingface.co/) e concordar com os Termos e Condições. Depois que isso for concluído, execute a célula abaixo que fornecerá um link para sua conta HuggingFace, onde você pode copiar o token. Depois de copiá-lo e colá-lo aqui, você poderá extrair e usar muitos dos modelos hospedados no hub HuggingFace.


In [8]:
notebook_login()

In [12]:
%%time

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "distil-whisper/distil-medium.en"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    torch_dtype=torch_dtype,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CPU times: user 1.11 s, sys: 72 ms, total: 1.18 s
Wall time: 3.05 s


Agora precisamos enviar o arquivo do podcast baixado para obter sua transcrição.

O que esse modelo faz é dividir o áudio em pedaços de 15 segundos e, em seguida, processar 16 desses pedaços em um lote. Isso acelera o processo significativamente em comparação com a implementação original do Whisper V2. Isso nos permite transcrever um podcast de 27 minutos em menos de 1 minuto!



---



**NOTA**:
- Esta etapa provavelmente levará mais tempo para ser concluída, dependendo da duração do episódio do podcast.
- Criamos este notebook com o tempo de execução da GPU habilitado e isso já vai acelerar as coisas. Na versão gratuita do Google Colab, o notebook provavelmente receberá uma GPU T4, que leva menos de um minuto para transcrever um episódio de 30 minutos.
- Se você tiver uma versão paga do Google Colab, poderá escolher uma GPU diferente, como V100 ou A100, para acelerar o trabalho.



---



In [13]:
%%time

podcast_transcript = pipe("/content/my_favourite_podcast.mp3")
print(podcast_transcript["text"])

 What does it take to move the needle on the world's toughest problems? On better heroes, we've sourced the globe for passionate individuals and visionary companies who are all on a mission to solve humanity's most urgent challenges. Like, can AI make the world a better place? How can we change our consumption habits to better serve the environment and what can we do to make our financial systems work for all? This series will convince you that humanity can save itself and our planet. Better Heroes is by EY and produced by human group media. Listen now, wherever you get your podcasts. The Economist. Hello and welcome to the Intelligence from The Economist. I'm your host Rosie Bloor. Every weekday we provide a fresh perspective on the event shaping your world. Champagne may be the drink of celebration, but wine collectors have long sniffed at the bubbly stuff. Now the global market is growing and Sotheby's is staging its first champagne-only auction. The market is looking rather sparkly

Para evitar a situação em que o notebook Colab encerra a sessão e você perde a variável Python que contém a transcrição, assim como para permitir testes mais rápidos de seções subsequentes, recomendamos que você crie uma variável local que contenha a transcrição do seu podcast.

In [14]:
my_transcript = podcast_transcript["text"]
my_transcript

" What does it take to move the needle on the world's toughest problems? On better heroes, we've sourced the globe for passionate individuals and visionary companies who are all on a mission to solve humanity's most urgent challenges. Like, can AI make the world a better place? How can we change our consumption habits to better serve the environment and what can we do to make our financial systems work for all? This series will convince you that humanity can save itself and our planet. Better Heroes is by EY and produced by human group media. Listen now, wherever you get your podcasts. The Economist. Hello and welcome to the Intelligence from The Economist. I'm your host Rosie Bloor. Every weekday we provide a fresh perspective on the event shaping your world. Champagne may be the drink of celebration, but wine collectors have long sniffed at the bubbly stuff. Now the global market is growing and Sotheby's is staging its first champagne-only auction. The market is looking rather sparkl

**Sugestões:**

1. Em vez de baixar e executar a transcrição nós mesmos, você também pode usar a versão hospedada do modelo OpenAI Whisper (pago). Para mais detalhes, você pode verificar a [documentação](https://platform.openai.com/docs/models/whisper).

2. O modelo Whisper não fornece identificação do falante, mas apenas transcreve o que foi dito. Em algumas situações, seria bom identificar quando há diferentes falantes no podcast e separar seus diálogos. Existem muitas [bibliotecas](https://github.com/m-bain/whisperX) que funcionam em cima do Whisper para habilitar isso.

# **II. Information Extraction**

## **2.1 Podcast Summary**

Começaremos criando um resumo do podcast. Queremos que seja conciso, mas ainda transmita a essência do episódio e tente chamar a atenção do usuário. Usaremos o modelo OpenAI `gpt-3.5-turbo` (pode-se usar outro como o `gpt-4o` ) para gerar este resumo apartir da transcrição gerada. Estamos pedindo ao LLM para analisar toda a transcrição que fornecemos e resumi-la para nós.

Precisamos usar as bibliotecas `openai` e `tiktoken`. A biblioteca openai é o pacote Python que nos permite fazer chamadas para a API. A biblioteca tiktoken nos permite determinar o número de tokens em nossa transcrição e isso nos dá uma indicação dos custos e também se precisaremos alterar o modelo que usamos para um com uma janela de contexto maior. Embora possamos usar a API e fazer chamadas diretamente, é muito mais fácil trabalhar com a biblioteca Python fornecida pela OpenAI chamada - `openai`.

In [15]:
client = OpenAI(api_key=userdata.get('MY_OPENAI_API_KEY'))

**Nota: Janela de contexto**

É importante entender o conceito de uma *contex window*. Este é o máximo do texto combinado que pode ser usado em uma chamada de API para os modelos de OpenAI. Não é apenas uma combinação do texto de entrada enviado ao modelo, pois também leva em conta a resposta de saída.

Também tenha em mente que a janela de contexto é medida em termos de **tokens** e não de palavras. Embora pudéssemos tratá-los como análogos, tecnicamente não é o mesmo, pois uma palavra pode realmente ser dividida em vários tokens.

É sempre recomendável usar o pacote tiktoken para determinar o número de tokens em seu texto.

In [16]:
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
print ("Number of tokens in input prompt ", len(enc.encode(my_transcript)))

Number of tokens in input prompt  5255


Como podemos ver no caso do episódio do podcast selecionado, o número de tokens é maior do que os 4096 tokens aceitos pelo padrão [modelo GPT-3.5-turbo](https://platform.openai.com/docs/models/gpt-3-5). O que isso significa é que temos que fazer uso do modelo de maior capacidade `gpt-3.5-turbo-16k` que tem um tamanho de contexto de 16.384 tokens. Vamos usar esse modelo para gerar o resumo do nosso podcast.

In [17]:
#montamos o nosso prompt

instructPrompt = """You are an expert copywriter responsible for publishing a newsletter with thousands of subscribers.
                    You listened to a great podcast and want to share a summary of it.
                    Please write the summary of this podcast, making sure to cover the  important topics in a concise way.
                    The transcript of the podcast is provided below: """

my_summary_prompt = instructPrompt + """****""" + my_transcript
my_summary_prompt

"You are an expert copywriter responsible for publishing a newsletter with thousands of subscribers.\n                    You listened to a great podcast and want to share a summary of it.\n                    Please write the summary of this podcast, making sure to cover the  important topics in a concise way.\n                    The transcript of the podcast is provided below: **** What does it take to move the needle on the world's toughest problems? On better heroes, we've sourced the globe for passionate individuals and visionary companies who are all on a mission to solve humanity's most urgent challenges. Like, can AI make the world a better place? How can we change our consumption habits to better serve the environment and what can we do to make our financial systems work for all? This series will convince you that humanity can save itself and our planet. Better Heroes is by EY and produced by human group media. Listen now, wherever you get your podcasts. The Economist. Hello 

In [18]:
chatOutput = client.chat.completions.create(model="gpt-3.5-turbo-16k",
                                            messages=[{"role": "system", "content": "You are a helpful assistant."},
                                                      {"role": "user", "content": my_summary_prompt}
                                                      ]
                                            )

In [19]:
podcastSummary = chatOutput.choices[0].message.content
podcastSummary

'Summary: The podcast discusses two main topics. \n\nThe first topic is the solar revolution and its impact on the global energy landscape. The podcast highlights that solar energy is growing rapidly and is on track to become the biggest source of electricity on the planet. The decreasing cost of solar power is a key factor driving its growth, with solar panels becoming increasingly affordable over time. The podcast emphasizes that the decline in solar panel prices has been significant, dropping from $100 per watt in 1974 to 11 cents per watt in recent years. However, limitations such as the intermittent nature of sunlight and the need for additional technology, like batteries, to store energy for use during non-sunny periods are mentioned.\n\nThe second topic revolves around the changing perception of champagne and its rising popularity among connoisseurs. The podcast mentions that champagne was historically perceived as a mass luxury product rather than a fine wine. However, the inte

## **2.2 Podcast Highlight**

In [20]:
instructPrompt = """
  You are a podcast editor and producer. You are provided with the transcript of a podcast episode and have to identify the 5 most significant moments in the podcast as highlights.
  - Each highlight needs to be a statement by one of the podcast guests
  - Each highlight has to be impactful and an important takeaway from this podcast episode
  - Each highlight must be concise and make listeners want to hear more about why the podcast guest said that
  - The highlights that you pick must be spread out throughout the episode

  Provide only the highlights and nothing else. Provide the full sentence of the highlight and format it as follows -

  - Highlight 1 of the podcast
  - Highlight 2 of the podcast
  - Highlight 3 of the podcast
  """

my_highlight_prompt = instructPrompt + """****""" + my_transcript
my_highlight_prompt

"\n  You are a podcast editor and producer. You are provided with the transcript of a podcast episode and have to identify the 5 most significant moments in the podcast as highlights.\n  - Each highlight needs to be a statement by one of the podcast guests\n  - Each highlight has to be impactful and an important takeaway from this podcast episode\n  - Each highlight must be concise and make listeners want to hear more about why the podcast guest said that\n  - The highlights that you pick must be spread out throughout the episode\n\n  Provide only the highlights and nothing else. Provide the full sentence of the highlight and format it as follows -\n\n  - Highlight 1 of the podcast\n  - Highlight 2 of the podcast\n  - Highlight 3 of the podcast\n  **** What does it take to move the needle on the world's toughest problems? On better heroes, we've sourced the globe for passionate individuals and visionary companies who are all on a mission to solve humanity's most urgent challenges. Like

In [21]:
chatOutput = client.chat.completions.create(model="gpt-3.5-turbo-16k",
                                            messages=[{"role": "system", "content": "You are a helpful assistant."},
                                                      {"role": "user", "content": my_highlight_prompt}
                                                      ]
                                            )
podcastHighlights = chatOutput.choices[0].message.content
podcastHighlights

'- Highlight 1 of the podcast: "Solar is growing so fast now that it\'s on track to generate more electricity than the world\'s nuclear power plants by 2026, its wind turbines by 2027, all of the hydroelectric dams by 2028, then it\'ll overtake electricity generated by gas in 2030 and coal in 2032. And at that point it will be the biggest source of electricity on the planet."\n- Highlight 2 of the podcast: "So the thing that is making all of this happen in truth is a giant manufacturing ecosystem in the harsh of industrial China, but there\'s slightly more to it than that. The real thing that\'s making solar go so fast is the nature of the technology itself."\n- Highlight 3 of the podcast: "These are batteries and very fortunately for solar panels and for humans really in general. The rate at which batteries are getting cheaper looks very similar to the rate of which batteries are getting cheaper, it looks very similar to the rate of which solar panels are getting cheaper. So if we\'re

## **2.3 Podcast Sentiment Analysis**

In [22]:
instructPrompt = """
  You are an experiment consultor of sentiment analysis. You are provided with the transcript of a podcast episode and have to identify if the
  text has in a global way positive, negative or neutral sentiments.

  Provide only the final sentiment label as well as the justification for this. Provide the answer formated as follows -

  - Sentiment of the podcast
  - Justification of sentiment label

  """

my_sentiment_prompt = instructPrompt + """****""" + my_transcript
my_sentiment_prompt

"\n  You are an experiment consultor of sentiment analysis. You are provided with the transcript of a podcast episode and have to identify if the\n  text has in a global way positive, negative or neutral sentiments.\n\n  Provide only the final sentiment label as well as the justification for this. Provide the answer formated as follows -\n\n  - Sentiment of the podcast\n  - Justification of sentiment label\n\n  **** What does it take to move the needle on the world's toughest problems? On better heroes, we've sourced the globe for passionate individuals and visionary companies who are all on a mission to solve humanity's most urgent challenges. Like, can AI make the world a better place? How can we change our consumption habits to better serve the environment and what can we do to make our financial systems work for all? This series will convince you that humanity can save itself and our planet. Better Heroes is by EY and produced by human group media. Listen now, wherever you get your

In [23]:
chatOutput = client.chat.completions.create(model="gpt-3.5-turbo-16k",
                                            messages=[{"role": "system", "content": "You are a helpful assistant."},
                                                      {"role": "user", "content": my_sentiment_prompt}
                                                      ]
                                            )
podcastSentiment = chatOutput.choices[0].message.content
podcastSentiment

"- Sentiment of the podcast: Positive\n- Justification of sentiment label: The overall sentiment of the podcast is positive because it discusses solutions to humanity's urgent challenges, such as using AI for good, changing consumption habits for the environment, and transforming financial systems. The podcast highlights the positive impact of solar energy and the potential for renewable energy to combat climate change. It also mentions the growing interest in champagne and the increasing recognition of its quality. Although there are brief mentions of potential challenges and limitations, the overall tone is optimistic and hopeful."

## **2.4 Formatting responses**

To better show our result, we format this as a JSON file

In [24]:
def process_podcast(url):
  output = {}
  output['podcast_url'] = podcast_url
  output['podcast_title'] = podcast_title
  output['episode_title'] = episode_title
  output['episode_image'] = episode_image
  output['episode_transcription'] = my_transcript
  output['episode_summary'] = podcastSummary
  output['episode_highlights'] = podcastHighlights
  output['episode_sentiment'] = podcastSentiment

  return output

# **III. Front-end App**

Nesta etapa final, usaremos o ChatGPT (a versão gratuita da web) como um assistente de programação para o desenvolvimento de uma interface simples mas funcional.

Abrimos um novo chat no ChatGPT e podemos colocar o seguinte prompt:



> Vamos trabalhar como parte de uma equipe de programação. Você é um renomado Software Engineer especialista em programação em Python e eu sou um Product Manager com forte background em desenvolvimento de software. Eu vou conferir cada linha de código que você for produzir e te ofereceei feedback para criar versões melhoradas de código e, assim, chegarmos a cumprir os requerimentos. Eu te oferecerei os requerimentos de maneira detalhada e, caso necessario, você pode solicitar as aclarações que considere necessarias. Quando você me ofereça código, por favor, além de comentarios preciso que você me explique o que você fez. Me confirme se minha instrução ficou clara respondendo apenas "Aguardando requerimentos, pronto para começar!"





Agora, podemos pedir a ChatGPT para criar uma página web de alto impacto em que se mostrem  os elementos que temos em JSON.

> Eu tenho um arquivo JSON com as seguintes informações: output['podcast_url'], output['podcast_title'], output['episode_title'], output['episode_image'],
output['episode_transcription'], output['episode_summary'], output['episode_highlights'], output['episode_sentiment']. Crie uma página web em formato HTML que apresente ao usuario final essas informações. Preciso que cada elemento JSON seja lido como uma variavel e logo seja montado o HTML. O referido HTML deve ser salvo como um string de uma variavel que logo será usado como componentes de uma app de gradio em Python.

In [25]:
def generate_html(podcast):
    # Extract data from JSON
    podcast_url = podcast.get('podcast_url', '')
    podcast_title = podcast.get('podcast_title', '')
    episode_title = podcast.get('episode_title', '')
    episode_image = podcast.get('episode_image', '')
    episode_transcription = podcast.get('episode_transcription', '')
    episode_summary = podcast.get('episode_summary', '')
    episode_highlights = podcast.get('episode_highlights', [])
    episode_sentiment = podcast.get('episode_sentiment', '')

    # Generate HTML string
    html_string = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>{podcast_title}</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                line-height: 1.6;
                margin: 0;
                padding: 0;
                background-color: #f4f4f4;
            }}
            .container {{
                max-width: 800px;
                margin: 20px auto;
                background: #fff;
                padding: 20px;
                box-shadow: 0 0 10px rgba(0,0,0,0.1);
                margin-bottom: 20px;
            }}
            h1, h2 {{
                color: #333;
            }}
            .episode-image {{
                max-width: 100%;
                height: auto;
            }}
            .podcast-url {{
                text-decoration: none;
                color: #007BFF;
            }}
            .podcast-url:hover {{
                text-decoration: underline;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <h1>{podcast_title}</h1>
            <a href="{podcast_url}" class="podcast-url" target="_blank">Listen to the Podcast</a>
            <h2>{episode_title}</h2>
            <img src="{episode_image}" class="episode-image" alt="Episode Image">

            <h3>Full Transcription</h3>
            <p>{episode_transcription}</p>

            <h3>Summary</h3>
            <p>{episode_summary}</p>

            <h3>Highlights</h3>
             <p>{episode_highlights}</p>

            <h3>Sentiment Analysis</h3>
            <p>{episode_sentiment}</p>
        </div>
    </body>
    </html>
    """
    return html_string



Agora, obtemos a informação desde o diretorio em que estamos salvando os JSON de nosso episodio do podcast. Aqui podemos ter um ou varios episodios descarregados.

In [27]:
def create_dict_from_json_files(folder_path):
    json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
    data_dict = {}

    for file_name in json_files:
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as file:
            podcast_info = json.load(file)
            podcast_name = podcast_info['podcast_title']
            # Process the file data as needed
            data_dict[podcast_name] = podcast_info

    return data_dict

available_podcast_info = create_dict_from_json_files('/content/')
available_podcast_info

{'The Intelligence from The Economist': {'podcast_url': 'https://access.acast.com/rss/d556eb54-6160-4c85-95f4-47d9f5216c49',
  'podcast_title': 'The Intelligence from The Economist',
  'episode_title': 'Empire of the sun: a solar power revolution',
  'episode_image': 'https://assets.pippa.io/shows/62e286a934d4d93d6587424a/1708532528162-b9341a1e28f72ef144e53cf135fce49e.jpeg',
  'episode_transcription': " What does it take to move the needle on the world's toughest problems? On better heroes, we've sourced the globe for passionate individuals and visionary companies who are all on a mission to solve humanity's most urgent challenges. Like, can AI make the world a better place? How can we change our consumption habits to better serve the environment and what can we do to make our financial systems work for all? This series will convince you that humanity can save itself and our planet. Better Heroes is by EY and produced by human group media. Listen now, wherever you get your podcasts. Th

In [28]:
def show_podcast_info(podcast_name):
  podcast_info = available_podcast_info[podcast_name]
  return generate_html(podcast_info)

In [29]:
def process_podcast_info(rss_url):
  podcast_info = process_podcast(rss_url)
  return generate_html(podcast_info)

In [30]:
with gr.Blocks() as demo:
    podcast_name = gr.Dropdown(available_podcast_info.keys(), label="podcast_name")
    show_podcast_button = gr.Button("Show Podcast Summary")
    podcast_url = gr.Textbox(label="podcast_url")
    process_podcast_button = gr.Button("Process Podcast")
    output= gr.HTML(label="podcast_info")
    show_podcast_button.click(fn=show_podcast_info, inputs=podcast_name, outputs=output, api_name="show_podcast_info")
    process_podcast_button.click(fn=process_podcast_info, inputs=podcast_url, outputs=output, api_name="process_podcast_info")

demo.launch(inbrowser=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://19421e70635b00a272.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
